CẤU HÌNH TẢI SENTINEL-1

In [ ]:
# Import thư viện và khởi tạo
import ee
import geemap
import os
import numpy as np
from datetime import datetime

# Khởi tạo Google Earth Engine với project ID
ee.Initialize(project='ee-bonglantrungmuoi')

# Cấu hình thời gian và tham số
start_date = '2024-04-01'
end_date = '2025-01-01'
ORBIT_DIRECTION = 'DESCENDING'
INSTRUMENT_MODE = 'IW'

# Tạo thư mục lưu dữ liệu
output_folder = './sentinel1_data'
os.makedirs(output_folder, exist_ok=True)

print(f"=== CẤU HÌNH TẢI SENTINEL-1 ===")
print(f"- Thời gian: {start_date} đến {end_date}")
print(f"- Orbit: {ORBIT_DIRECTION}")
print(f"- Instrument: {INSTRUMENT_MODE}")
print(f"- Thư mục lưu: {output_folder}")
print(f"- Sử dụng: geemap cho file lớn")

TẢI SHAPEFILE VÀ TẠO COLLECTION

In [ ]:
# Đọc shapefile và tạo Sentinel-1 collection
print("=== TẢI SHAPEFILE VÀ TẠO COLLECTION ===")

# Sử dụng shapefile Lâm Đồng từ GEE Asset
lamdong_asset = ee.FeatureCollection('projects/ee-bonglantrungmuoi/assets/lamdong')
lamdong_geometry = lamdong_asset.geometry()

# Hiển thị thông tin về shapefile
print(f"✓ Đã tải shapefile từ GEE Asset")
print(f"  - Số features: {lamdong_asset.size().getInfo()}")

# Tạo Sentinel-1 collection
sentinel1_collection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                        .filterBounds(lamdong_geometry)
                        .filterDate(start_date, end_date)
                        .filter(ee.Filter.eq('orbitProperties_pass', ORBIT_DIRECTION))
                        .filter(ee.Filter.eq('instrumentMode', INSTRUMENT_MODE))
                        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')))

# Thông tin collection
collection_size = sentinel1_collection.size().getInfo()
print(f"✓ Đã tạo Sentinel-1 collection")
print(f"  - Tổng số ảnh: {collection_size}")

# Thông tin ảnh đầu tiên
if collection_size > 0:
    first_image = ee.Image(sentinel1_collection.first())
    first_date = ee.Date(first_image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    print(f"  - Ảnh đầu tiên: {first_date}")
    
    # Thống kê theo năm
    for year in range(2022, 2025):
        year_collection = sentinel1_collection.filter(ee.Filter.calendarRange(year, year, 'year'))
        year_count = year_collection.size().getInfo()
        print(f"  - Năm {year}: {year_count} ảnh")
else:
    print("⚠️ Không tìm thấy ảnh nào!")

THIẾT LẬP FUNCTIONS TẢI DỮ LIỆU

In [ ]:
# Functions tải dữ liệu với geemap
print("=== THIẾT LẬP FUNCTIONS TẢI DỮ LIỆU ===")

def download_monthly_composite(year, month, resolution=10):
    """
    Tải composite tháng với geemap
    
    Args:
        year: Năm
        month: Tháng  
        resolution: Độ phân giải (m), mặc định 10m
    """
    start_month = ee.Date.fromYMD(year, month, 1)
    end_month = start_month.advance(1, 'month')
    
    # Lọc ảnh theo tháng
    monthly_collection = sentinel1_collection.filterDate(start_month, end_month)
    
    # Kiểm tra có dữ liệu không
    count = monthly_collection.size().getInfo()
    if count == 0:
        print(f"❌ Không có dữ liệu cho {year}-{month:02d}")
        return None
    
    print(f"📊 Tháng {year}-{month:02d}: {count} ảnh gốc")
    
    # Tạo composite và clip
    composite = (monthly_collection
                .select(['VV', 'VH'])
                .median()
                .clip(lamdong_geometry))
    
    # Tạo tên file
    filename = f'S1_Lamdong_{year}_{month:02d}_res{resolution}m.tif'
    filepath = os.path.join(output_folder, filename)
    
    try:
        print(f"🔄 Đang tải {filename}...")
        
        # Sử dụng geemap để tải file lớn (bỏ file_per_band parameter)
        geemap.download_ee_image(
            image=composite,
            filename=filepath,
            scale=resolution,
            region=lamdong_geometry,
            crs='EPSG:4326'
        )
        
        # Kiểm tra file size
        if os.path.exists(filepath):
            file_size = os.path.getsize(filepath) / (1024*1024)  # MB
            print(f"✅ Đã lưu: {filename} ({file_size:.1f} MB)")
            return filepath
        else:
            print(f"❌ Không tạo được file: {filename}")
            return None
        
    except Exception as e:
        print(f"❌ Lỗi tải {year}-{month:02d}: {str(e)}")
        return None

def download_individual_image(index, resolution=10):
    """
    Tải ảnh riêng lẻ theo index
    
    Args:
        index: Chỉ số ảnh trong collection
        resolution: Độ phân giải (m)
    """
    try:
        image = ee.Image(sentinel1_collection.toList(1, index).get(0))
        date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd').getInfo()
        
        processed = (image
                    .select(['VV', 'VH'])
                    .clip(lamdong_geometry))
        
        filename = f'S1_Lamdong_{date}_res{resolution}m.tif'
        filepath = os.path.join(output_folder, filename)
        
        print(f"🔄 Đang tải {filename}...")
        
        # Bỏ file_per_band parameter
        geemap.download_ee_image(
            image=processed,
            filename=filepath,
            scale=resolution,
            region=lamdong_geometry,
            crs='EPSG:4326'
        )
        
        if os.path.exists(filepath):
            file_size = os.path.getsize(filepath) / (1024*1024)
            print(f"✅ Đã lưu: {filename} ({file_size:.1f} MB)")
            return filepath
        else:
            print(f"❌ Không tạo được file: {filename}")
            return None
        
    except Exception as e:
        print(f"❌ Lỗi tải ảnh {index}: {str(e)}")
        return None

def estimate_file_size(resolution=10):
    """Ước tính kích thước file"""
    try:
        sample_image = ee.Image(sentinel1_collection.first())
        pixel_count = (sample_image.select('VV')
                      .reduceRegion(
                          reducer=ee.Reducer.count(),
                          geometry=lamdong_geometry,
                          scale=resolution,
                          maxPixels=1e9
                      ).get('VV').getInfo())
        
        # Ước tính: 2 bands × 4 bytes/pixel (Float32)
        size_mb = (pixel_count * 2 * 4) / (1024 * 1024)
        
        print(f"📏 Ước tính kích thước file (độ phân giải {resolution}m):")
        print(f"   - Số pixel: {pixel_count:,}")
        print(f"   - Kích thước: {size_mb:.1f} MB ({size_mb/1024:.2f} GB)")
        
        return size_mb
        
    except Exception as e:
        print(f"❌ Không thể ước tính: {str(e)}")
        return 0

print("✅ Đã thiết lập functions tải dữ liệu với geemap!")

TẢI COMPOSITE THÁNG VV VH (TRUNG BÌNH THÁNG)

In [ ]:
print("=== TẢI ẢNH TRUNG BÌNH THÁNG TỪNG THÁNG ===")
print("Thời gian: Tháng 1/2022 → Tháng 12/2024")
print("Bands: VV, VH")

# Cấu hình
RESOLUTION = 10  # Độ phân giải 10m
total_months = 3 * 12  # 3 năm × 12 tháng = 36 tháng

# Ước tính kích thước file
print(f"\n🔍 Ước tính kích thước với độ phân giải {RESOLUTION}m:")
estimated_size = estimate_file_size(RESOLUTION)

print(f"📊 Tổng quan:")
print(f"   - Số tháng cần tải: {total_months} tháng")
print(f"   - Kích thước ước tính/file: {estimated_size:.1f} MB")
print(f"   - Tổng dung lượng ước tính: {(estimated_size * total_months):.1f} MB ({(estimated_size * total_months)/1024:.1f} GB)")

# Xác nhận trước khi tải
if estimated_size > 500:  # Nếu file > 500MB
    print(f"\n⚠️  Mỗi file sẽ rất lớn ({estimated_size:.0f} MB)")
    confirm = input("Bạn có muốn tiếp tục? (y/N): ")
    if confirm.lower() != 'y':
        print("❌ Hủy bỏ quá trình tải")
        exit()

print(f"\n🚀 BẮT ĐẦU TẢI {total_months} COMPOSITE THÁNG...")

downloaded_files = []
failed_months = []
count = 0

# Tải từng tháng từ 2022-2024
for year in range(2022, 2025):  # 2022, 2023, 2024
    for month in range(1, 13):  # Tháng 1-12
        count += 1
        
        print(f"\n[{count}/{total_months}] Tháng {year}-{month:02d}")
        
        try:
            filepath = download_monthly_composite(year, month, RESOLUTION)
            
            if filepath:
                downloaded_files.append(filepath)
                print(f"✅ Thành công: {os.path.basename(filepath)}")
            else:
                failed_months.append(f"{year}-{month:02d}")
                print(f"❌ Thất bại: {year}-{month:02d}")
                
        except Exception as e:
            failed_months.append(f"{year}-{month:02d}")
            print(f"❌ Lỗi {year}-{month:02d}: {str(e)}")
        
        # Hiển thị tiến trình
        progress = (count / total_months) * 100
        print(f"📈 Tiến trình: {progress:.1f}% ({count}/{total_months})")

# Tổng kết
print(f"\n{'='*50}")
print(f"🎉 HOÀN TẤT QUÁ TRÌNH TẢI!")
print(f"{'='*50}")

if downloaded_files:
    total_size = sum(os.path.getsize(f) for f in downloaded_files) / (1024*1024)
    
    print(f"✅ Thành công:")
    print(f"   - Đã tải: {len(downloaded_files)}/{total_months} file")
    print(f"   - Tổng dung lượng: {total_size:.1f} MB ({total_size/1024:.1f} GB)")
    print(f"   - Thư mục: {output_folder}")
    
    # Thống kê theo năm
    for year in range(2022, 2025):
        year_files = [f for f in downloaded_files if f"_{year}_" in f]
        print(f"   - Năm {year}: {len(year_files)} file")

if failed_months:
    print(f"\n❌ Thất bại ({len(failed_months)} tháng):")
    for month in failed_months:
        print(f"   - {month}")
    print(f"\n💡 Có thể thử lại các tháng thất bại bằng cách chạy:")
    print(f"   download_monthly_composite(year, month, {RESOLUTION})")

print(f"\n📋 Danh sách file đã tải:")
for i, filepath in enumerate(downloaded_files, 1):
    filename = os.path.basename(filepath)
    size = os.path.getsize(filepath) / (1024*1024)
    print(f"   {i:2d}. {filename} ({size:.1f} MB)")

print(f"\n🔍 Sử dụng cell 5 để kiểm tra chi tiết và quản lý file!")

KIỂM TRA VÀ QUẢN LÝ KẾT QUẢ

In [ ]:
print("=== KIỂM TRA KẾT QUẢ TẢI ===")

def check_downloaded_files():
    """Kiểm tra các file đã tải với thông tin chi tiết"""
    if not os.path.exists(output_folder):
        print("❌ Thư mục dữ liệu chưa tồn tại")
        return []
    
    files = [f for f in os.listdir(output_folder) if f.endswith('.tif')]
    
    if not files:
        print("📂 Thư mục trống - chưa có file nào")
        return []
    
    # Sắp xếp files theo tên
    files.sort()
    
    print(f"📊 Đã tải {len(files)} file:")
    print(f"📁 Thư mục: {output_folder}")
    
    total_size = 0
    monthly_files = []
    individual_files = []
    
    for i, file in enumerate(files, 1):
        filepath = os.path.join(output_folder, file)
        size = os.path.getsize(filepath) / (1024*1024)  # MB
        total_size += size
        
        # Phân loại file
        if "_res" in file and ("_01_" in file or "_02_" in file):
            monthly_files.append((file, size))
        else:
            individual_files.append((file, size))
        
        print(f"   {i:2d}. {file} ({size:.1f} MB)")
    
    print(f"\n📈 Tổng quan:")
    print(f"   - Composite tháng: {len(monthly_files)} file")
    print(f"   - Ảnh riêng lẻ: {len(individual_files)} file")
    print(f"   - Tổng dung lượng: {total_size:.1f} MB ({total_size/1024:.2f} GB)")
    
    return files

def view_file_info(filename):
    """Xem thông tin chi tiết của file"""
    filepath = os.path.join(output_folder, filename)
    
    if not os.path.exists(filepath):
        print(f"❌ File không tồn tại: {filename}")
        return
    
    try:
        import rasterio
        
        with rasterio.open(filepath) as src:
            print(f"📄 Thông tin file: {filename}")
            print(f"   - Kích thước: {src.width} x {src.height} pixels")
            print(f"   - Số bands: {src.count}")
            print(f"   - CRS: {src.crs}")
            print(f"   - Bounds: {src.bounds}")
            print(f"   - Data type: {src.dtypes}")
            
            # Đọc một phần dữ liệu để thống kê
            vv_sample = src.read(1, window=((0, min(100, src.height)), (0, min(100, src.width))))
            vh_sample = src.read(2, window=((0, min(100, src.height)), (0, min(100, src.width))))
            
            print(f"   - VV range: {vv_sample.min():.2f} to {vv_sample.max():.2f} dB")
            print(f"   - VH range: {vh_sample.min():.2f} to {vh_sample.max():.2f} dB")
            
    except ImportError:
        print(f"💡 Cài đặt rasterio để xem thông tin chi tiết: pip install rasterio")
        file_size = os.path.getsize(filepath) / (1024*1024)
        print(f"📄 {filename}: {file_size:.1f} MB")
    except Exception as e:
        print(f"❌ Lỗi đọc file: {str(e)}")

def clean_output_folder():
    """Xóa tất cả file trong thư mục output"""
    if not os.path.exists(output_folder):
        print("❌ Thư mục không tồn tại")
        return
    
    files = [f for f in os.listdir(output_folder) if f.endswith('.tif')]
    
    if not files:
        print("📂 Thư mục đã trống")
        return
    
    confirm = input(f"⚠️  Xóa {len(files)} file? (y/N): ")
    if confirm.lower() == 'y':
        for file in files:
            os.remove(os.path.join(output_folder, file))
        print(f"🗑️  Đã xóa {len(files)} file")
    else:
        print("❌ Hủy bỏ")

# Kiểm tra kết quả hiện tại
files = check_downloaded_files()

if files:
    print(f"\n💡 Sử dụng:")
    print(f"   - view_file_info('filename.tif') để xem chi tiết file")
    print(f"   - clean_output_folder() để xóa tất cả file")
    
    # Hiển thị thông tin file đầu tiên
    if len(files) > 0:
        print(f"\n🔍 Thông tin file đầu tiên:")
        view_file_info(files[0])